In [1]:
#Импортируем необходимые библиотеки
import csv
import requests
import time
import random

from bs4 import BeautifulSoup

In [2]:
#Функция, которая получает контент страницы по ее url
def get_page_content(url):
    result = requests.get(url)
    
    if result.status_code == 200:
        return result.text
    else:
        return None

In [3]:
#функция, которая парсит необходимые данные со страницы по определенным тегам
def parse_content(page):
    soup = BeautifulSoup(page, 'lxml') # Изпользуем BeautifulSoupe
    
    title = soup.find('h1', class_="b-title b-title_type_h1 b-title_no-margin b-text-inline").get_text().strip()
    
    year = title.split(',')[1].split(' ')[1]
    
    price = soup.find('div', class_="css-1dpi5rw e162wx9x0")
    
    if price == None:
        price = soup.find('div', class_="css-4mziby e162wx9x0")
        
    price = price.get_text().replace('q', '').strip()
    price = "".join(price.split())
    
    model = title.split(',')[0].split(' ')[2]
    
    
    engine = soup.find('span', class_='b-text-gray', text='Двигатель:')
    if engine != None:
        fuelType = engine.parent.get_text().split(' ')[1][:-1]
        engineDisplacement = engine.parent.get_text().split(' ')[2]
    
    enginePower = soup.find('div', class_="b-triggers__text")
    if enginePower != None:
        enginePower = enginePower.get_text().strip().split(' ')[0]
    
    vehicleTransmission = soup.find('span', class_='b-text-gray', text='Трансмиссия:')
    if vehicleTransmission != None:
        vehicleTransmission = vehicleTransmission.parent.get_text().split(' ')[1]
        
    
    gearing = soup.find('span', class_='b-text-gray', text='Привод:')
    if gearing != None:
        gearing = gearing.parent.get_text().split(' ')[1]
    
    color = soup.find('span', class_='b-text-gray', text='Цвет:')
    if color != None:
        color = color.parent.get_text().split(' ')[1]
    
    mileage = soup.find('span', class_='b-text-gray', text='Пробег, км:')
    if mileage != None:
        mileage = mileage.parent.get_text().split(' ')[2]
    
    rudder = soup.find('span', class_='b-text-gray', text='Руль:')
    if rudder != None:
        rudder = rudder.parent.get_text().split(' ')[1]
    
    generation = soup.find('span', class_='b-text-gray', text='Поколение:')
    if generation != None:
        generation = generation.parent.get_text().split(' ')[1]
        
    equipment = soup.find('span', class_='b-text-gray', text='Комплектация:')
    if equipment != None:
        equipment = equipment.parent.get_text().split(':')[1].strip()
        
    bodyType = soup.find('span', class_='b-text-gray', text='Тип кузова:')
    if bodyType != None:
        bodyType = bodyType.parent.get_text().split(' ')[2].strip()
        
    
    vin = soup.find_all('span', class_="css-14reqj3 e1wvjnck0")
    owners = None
    if len(vin) >= 2:
        owners = vin[1].get_text()[0]
    
    return (bodyType, 'BMW', color, fuelType, generation, model, None, year, None, vehicleTransmission, 
            engineDisplacement, enginePower, None, mileage, equipment, gearing, rudder, None, 
            owners, None, None, None, None, price)

In [5]:
#Записываем полученные данные в csv-файл
with open("bmws.csv", "wt", newline='') as output_file:
    writer = csv.writer(output_file, delimiter=",")
    writer.writerow(('bodyType','brand','color','fuelType','modelDate','name','numberOfDoors','productionDate',
                     'vehicleConfiguration','vehicleTransmission','engineDisplacement','enginePower','description',
                     'mileage','Комплектация','Привод','Руль','Состояние','Владельцы','ПТС','Таможня','Владение','id'))
    
    with open('bmw_links.txt') as links_file:
        for link in links_file:
            print('Parsing: ', link)

            page_content = get_page_content(link)
            
            if page_content == None:
                continue
            
            parsed_content = None
            
            try_num = 0;
            while parsed_content == None:
                
                if try_num >= 15:
                    break;
                
                try:
                    try_num += 1
                    page_content = get_page_content(link)
                    parsed_content = parse_content(page_content)
                except:
                    pass
                finally:
                    print("Try number: ", try_num)
            
            if try_num >= 15:
                continue
                    
            print(parsed_content)

            writer.writerow(parsed_content)

Parsing:  https://moscow.drom.ru/bmw/m5/38583565.html

Try number:  1
('седан', 'BMW', 'серебристый', 'бензин', '6', 'M5', None, '2019', None, 'автомат', '4.4', '625', None, '3600', '4.4 AT xDrive Competition', '4WD', 'левый', None, '1', None, None, None, None, '7600000')
Parsing:  https://moscow.drom.ru/bmw/3-series/38711120.html

Try number:  1
('седан', 'BMW', 'белый', 'бензин', '6', '3-Series', None, '2017', None, 'автомат', '2.0', '184', None, '42000', '320i AT xDrive M Sport', '4WD', 'левый', None, '3', None, None, None, None, '1300000')
Parsing:  https://moscow.drom.ru/bmw/x1/38710873.html

Try number:  1
(None, 'BMW', 'коричневый', 'дизель', '1', 'X1', None, '2012', None, 'автомат', '2.0', '177', None, '121000', None, '4WD', 'левый', None, '1', None, None, None, None, '799000')
Parsing:  https://moscow.drom.ru/bmw/m3/38042941.html

Try number:  1
('купе', 'BMW', 'белый', 'бензин', '4', 'M3', None, '2008', None, 'механика', '4.0', '420', None, '179000', '4.0 MT', 'задний', 'левы

Try number:  1
('хэтчбек', 'BMW', 'красный', 'бензин', None, '1-Series', None, '2010', None, 'автомат', '1.6', '115', None, '123530', None, 'задний', 'левый', None, '2', None, None, None, None, '459000')
Parsing:  https://moscow.drom.ru/bmw/3-series/38703058.html

Try number:  1
('седан', 'BMW', 'белый', 'бензин', '6', '3-Series', None, '2014', None, 'автомат', '1.6', '136', None, '75318', None, 'задний', 'левый', None, '1', None, None, None, None, '899000')
Parsing:  https://moscow.drom.ru/bmw/x1/38703056.html

Try number:  1
(None, 'BMW', 'коричневый', 'бензин', '1', 'X1', None, '2013', None, 'автомат', '2.0', '184', None, '196552', None, '4WD', 'левый', None, '4', None, None, None, None, '765000')
Parsing:  https://moscow.drom.ru/bmw/5-series/38703049.html

Try number:  1
('седан', 'BMW', 'белый', 'бензин', '6', '5-Series', None, '2012', None, 'автомат', '3.0', '306', None, '133000', None, '4WD', 'левый', None, '1', None, None, None, None, '1457000')
Parsing:  https://moscow.drom.ru

Try number:  1
(None, 'BMW', 'белый', 'бензин', '1', 'X2', None, '2019', None, 'автомат', '1.5', '140', None, '17602', 'sDrive 18i DCT Base', 'передний', 'левый', None, '1', None, None, None, None, '1842000')
Parsing:  https://moscow.drom.ru/bmw/5-series/38700984.html

Try number:  1
('седан', 'BMW', 'белый', 'дизель', '7', '5-Series', None, '2019', None, 'автомат', '3.0', '249', None, '22096', None, '4WD', 'левый', None, '1', None, None, None, None, '3550000')
Parsing:  https://moscow.drom.ru/bmw/x5/38700957.html

Try number:  1
(None, 'BMW', 'белый', 'дизель', '4', 'X5', None, '2019', None, 'автомат', '3.0', '249', None, '25200', None, '4WD', 'левый', None, '1', None, None, None, None, '5100000')
Parsing:  https://moscow.drom.ru/bmw/3-series/38700955.html

Try number:  1
('седан', 'BMW', 'черный', 'бензин', '6', '3-Series', None, '2017', None, 'автомат', '2.0', '184', None, '64071', None, 'задний', 'левый', None, '1', None, None, None, None, '1549000')
Parsing:  https://moscow.drom.r

Try number:  1
('седан', 'BMW', 'белый', 'дизель', '7', '5-Series', None, '2018', None, 'автомат', '2.0', '190', None, '32749', None, '4WD', 'левый', None, '1', None, None, None, None, '2570000')
Parsing:  https://moscow.drom.ru/bmw/7-series/36292248.html

Try number:  1
(None, 'BMW', 'черный', 'бензин', '5', '7-Series', None, '2014', None, 'автомат', '3.0', '320', None, '93773', '740Li AT xDrive', '4WD', 'левый', None, '3', None, None, None, None, '1630000')
Parsing:  https://moscow.drom.ru/bmw/1-series/37219664.html

Try number:  1
('хэтчбек', 'BMW', 'белый', 'бензин', '2', '1-Series', None, '2017', None, 'автомат', '1.5', '136', None, '66693', None, 'задний', 'левый', None, '1', None, None, None, None, '1149000')
Parsing:  https://moscow.drom.ru/bmw/4-series/37245692.html

Try number:  1
('купе', 'BMW', 'белый', 'дизель', '1', '4-Series', None, '2016', None, 'автомат', '2.0', '190', None, '77796', '420d AT xDrive', '4WD', 'левый', None, '3', None, None, None, None, '1820000')
Parsin

Try number:  1
(None, 'BMW', 'синий', 'дизель', '1', '6-Series', None, '2018', None, 'автомат', '3.0', '249', None, '26056', None, '4WD', 'левый', None, '1', None, None, None, None, '3297000')
Parsing:  https://moscow.drom.ru/bmw/5-series/38690023.html

Try number:  1
('седан', 'BMW', 'белый', 'дизель', '7', '5-Series', None, '2018', None, 'автомат', '3.0', '249', None, '27819', None, '4WD', 'левый', None, '3', None, None, None, None, '3199000')
Parsing:  https://moscow.drom.ru/bmw/5-series/38689965.html

Try number:  1
('универсал', 'BMW', 'серый', 'бензин', '6', '5-Series', None, '2014', None, 'автомат', '2.0', '245', None, '146048', None, '4WD', 'левый', None, '1', None, None, None, None, '1449000')
Parsing:  https://moscow.drom.ru/bmw/6-series/38689939.html

Try number:  1
('седан', 'BMW', 'синий', 'дизель', '3', '6-Series', None, '2012', None, 'автомат', '3.0', '313', None, '56240', '640d AT', 'задний', 'левый', None, '3', None, None, None, None, '2169000')
Parsing:  https://mosco

...

Try number:  1
(None, 'BMW', 'черный', 'дизель', '6', '7-Series', None, '2018', None, 'автомат', '3.0', '320', None, '25764', None, '4WD', 'левый', None, '1', None, None, None, None, '4004793')
Parsing:  https://moscow.drom.ru/bmw/x4/37624784.html

Try number:  1
(None, 'BMW', 'черный', 'дизель', '2', 'X4', None, '2019', None, 'автомат', '3.0', '326', None, '1521', 'xDrive M40d AT Base', '4WD', 'левый', None, '1', None, None, None, None, '4852000')
Parsing:  https://moscow.drom.ru/bmw/7-series/37624757.html

Try number:  1
Try number:  2
(None, 'BMW', 'черный', 'дизель', None, '7-Series', None, '2018', None, 'автомат', '3.0', '249', None, '19556', None, '4WD', 'левый', None, '1', None, None, None, None, '3674688')
Parsing:  https://moscow.drom.ru/bmw/7-series/36524548.html

Try number:  1
(None, 'BMW', 'черный', 'дизель', None, '7-Series', None, '2018', None, 'автомат', '3.0', '249', None, '19681', None, '4WD', 'левый', None, '1', None, None, None, None, '3683142')
Parsing:  https://mo

Try number:  1
('хэтчбек', 'BMW', 'черный', 'бензин', '1', '1-Series', None, '2011', None, 'автомат', '1.6', '116', None, '146988', '116i AT Base', 'задний', 'левый', None, '1', None, None, None, None, '423000')
Parsing:  https://moscow.drom.ru/bmw/3-series/37421919.html

Try number:  1
('универсал', 'BMW', 'черный', 'бензин', '5', '3-Series', None, '2008', None, 'автомат', '2.5', '218', None, '148000', None, '4WD', 'левый', None, '5', None, None, None, None, '570000')
Parsing:  https://moscow.drom.ru/bmw/x1/38400214.html

Try number:  1
(None, 'BMW', 'белый', 'бензин', '2', 'X1', None, '2015', None, 'автомат', '2.0', '192', None, '58157', 'sDrive 20i AT', 'передний', 'левый', None, '2', None, None, None, None, '1439000')
Parsing:  https://moscow.drom.ru/bmw/x1/38400080.html

Try number:  1
(None, 'BMW', 'серый', 'бензин', '2', 'X1', None, '2018', None, 'автомат', '2.0', '192', None, '26543', None, '4WD', 'левый', None, '1', None, None, None, None, '1860000')
Parsing:  https://moscow.d

Try number:  1
(None, 'BMW', 'черный', 'бензин', '1', 'X7', None, '2019', None, 'автомат', '3.0', '340', None, '1691', None, '4WD', 'левый', None, '1', None, None, None, None, '6200000')
Parsing:  https://moscow.drom.ru/bmw/x5/38385426.html

Try number:  1
(None, 'BMW', 'синий', 'бензин', '1', 'X5', None, '2004', None, 'автомат', '4.8', '360', None, '300000', '4.8si AT xDrive', '4WD', 'левый', None, '6', None, None, None, None, '950000')
Parsing:  https://moscow.drom.ru/bmw/5-series/38383806.html

Try number:  1
('седан', 'BMW', None, 'бензин', '5', '5-Series', None, '2004', None, 'автомат', '4.4', '333', None, None, None, 'задний', 'левый', None, '5', None, None, None, None, '680000')
Parsing:  https://moscow.drom.ru/bmw/5-series/38383538.html

Try number:  1
('седан', 'BMW', None, 'бензин', None, '5-Series', None, '2005', None, 'автомат', '2.5', '176', None, '330000', None, 'задний', 'левый', None, '8', None, None, None, None, '450000')
Parsing:  https://moscow.drom.ru/bmw/3-series/3

Try number:  1
(None, 'BMW', 'черный', 'дизель', '5', '7-Series', None, '2011', None, 'автомат', '3.0', '306', None, '134207', '740d AT', 'задний', 'левый', None, '4', None, None, None, None, '1299000')
Parsing:  https://moscow.drom.ru/bmw/5-series/38369160.html

Try number:  1
('седан', 'BMW', None, 'бензин', '6', '5-Series', None, '2011', None, 'автомат', '3.0', '306', None, '208000', None, '4WD', 'левый', None, '7', None, None, None, None, '950000')
Parsing:  https://moscow.drom.ru/bmw/3-series/38368237.html

Try number:  1
('седан', 'BMW', 'серый', 'бензин', '4', '3-Series', None, '2002', None, 'автомат', '2.5', '192', None, '231000', None, '4WD', 'левый', None, '5', None, None, None, None, '374900')
Parsing:  https://moscow.drom.ru/bmw/5-series/38367986.html

Try number:  1
('седан', 'BMW', 'черный', 'дизель', '6', '5-Series', None, '2014', None, 'автомат', '2.0', '184', None, '184211', None, 'задний', 'левый', None, '4', None, None, None, None, '1117000')
Parsing:  https://moscow

Try number:  1
('седан', 'BMW', 'синий', 'бензин', None, '3-Series', None, '2019', None, 'автомат', '2.0', '184', None, '2000', None, '4WD', 'левый', None, '1', None, None, None, None, '2990000')
Parsing:  https://moscow.drom.ru/bmw/8-series/36985557.html

Try number:  1
('купе', 'BMW', 'оранжевый', 'бензин', '2', '8-Series', None, '2019', None, 'автомат', '4.4', '530', None, '1057', 'M850i AT xDrive Base', '4WD', 'левый', None, '1', None, None, None, None, '8120000')
Parsing:  https://moscow.drom.ru/bmw/x5/36985558.html

Try number:  1
(None, 'BMW', 'черный', 'дизель', '4', 'X5', None, '2019', None, 'автомат', '3.0', '249', None, '1047', None, '4WD', 'левый', None, '1', None, None, None, None, '5449999')
Parsing:  https://moscow.drom.ru/bmw/x6/33287469.html

Try number:  1
(None, 'BMW', 'черный', 'бензин', None, 'X6', None, '2018', None, 'автомат', '4.4', '575', None, '28000', None, '4WD', 'левый', None, '1', None, None, None, None, '5899000')
Parsing:  https://moscow.drom.ru/bmw/x5/3

# Попытки спарсить ссылки внутри страницы

In [28]:
import csv
import requests
import time
import random

from bs4 import BeautifulSoup

In [29]:
def get_page_content(url):
    result = requests.get(url)
           
    if result.status_code == 200:
        return result.text
    else:
        return None

In [30]:
def parse_content(page):
    soup = BeautifulSoup(page, 'lxml')
        
    title = soup.find('h1', class_="b-title b-title_type_h1 b-title_no-margin b-text-inline").get_text().strip()
    
    year = title.split(',')[1].split(' ')[1]
    
    price = soup.find('div', class_="css-7ocpws e162wx9x0")
    
    if price == None:
        price = soup.find('div', class_="css-7ocpws e162wx9x0")
        
    price = price.get_text().replace('q', '').strip()
    price = "".join(price.split())
        
    model = title.split(',')[0].split(' ')[2]
    
    
    engine = soup.find('span', class_='b-text-gray', text='Двигатель:')
    if engine != None:
        fuelType = engine.parent.get_text().split(' ')[1][:-1]
        engineDisplacement = engine.parent.get_text().split(' ')[2]
    
    enginePower = soup.find('div', class_="b-triggers__text")
    if enginePower != None:
        enginePower = enginePower.get_text().strip().split(' ')[0]
    
    vehicleTransmission = soup.find('span', class_='b-text-gray', text='Трансмиссия:')
    if vehicleTransmission != None:
        vehicleTransmission = vehicleTransmission.parent.get_text().split(' ')[1]
        
    gearing = soup.find('span', class_='b-text-gray', text='Привод:')
    if gearing != None:
        gearing = gearing.parent.get_text().split(' ')[1]
    
    color = soup.find('span', class_='b-text-gray', text='Цвет:')
    if color != None:
        color = color.parent.get_text().split(' ')[1]
    
    mileage = soup.find('span', class_='b-text-gray', text='Пробег, км:')
    if mileage != None:
        mileage = mileage.parent.get_text().split(' ')[2]
    
    rudder = soup.find('span', class_='b-text-gray', text='Руль:')
    if rudder != None:
        rudder = rudder.parent.get_text().split(' ')[1]
    
    generation = soup.find('span', class_='b-text-gray', text='Поколение:')
    if generation != None:
        generation = generation.parent.get_text().split(' ')[1]
        
    equipment = soup.find('span', class_='b-text-gray', text='Комплектация:')
    equipment_link = None
    if equipment != None:
        equipment_link = equipment.parent.find('a', class_ = 'b-link')['href']
        
    doors = None
    equipment = None
    name = None 
    
    if equipment_link != None:    
        doors, name, equipment = parse_content_specifications(get_page_content(equipment_link))   
   
      
    bodyType = soup.find('span', class_='b-text-gray', text='Тип кузова:')
    if bodyType != None:
        bodyType = bodyType.parent.get_text().split(' ')[2].strip()
        
    state = soup.find('span', class_='b-text-gray', text='Особые отметки:')
    if state != None:
        state = state.parent.get_text().split(':')[1].strip()
    
    vin = soup.find_all('span', class_="css-14reqj3 e1wvjnck0")
    owners = None
    if len(vin) >= 2:
        owners = vin[1].get_text()[0]
        
    #description = soup.find('span', class_='b-text-gray', text='Дополнительно:')
    #if description != None:
        #description = description.parent.get_text()
        
    return (bodyType, 'BMW', color, fuelType, generation, name, doors, year, None, vehicleTransmission, 
            engineDisplacement, enginePower, None, mileage, equipment, gearing, rudder, state, 
            owners, None, None, None, price, model)
    

In [31]:
def parse_content_specifications(page):
    soup = BeautifulSoup(page, 'lxml')
    
    #Количество дверей
    doors = soup.find('div', class_="b-model-specs__label", text='Кол-во дверей')
    if doors != None:
        doors = doors.parent.find('div', class_="b-model-specs__text").get_text().split(' ')[0]
        
    #Комплектация:
    name_compl = soup.find('td', text='Название комплектации')
    name = None
    if name_compl != None:
        name_compl = name_compl.parent.find_all('td')
        name = name_compl[1].get_text().strip()
        
    car_compl = []
    compls = soup.find_all('tr', class_="b-table__row b-table__row_padding_l-r-size-xs b-table__row_cols_2 b-table__row_border_bottom b-table__row_border_light b-table__row_padding_t-b-size-s b-table_align_top")
    for compl in compls:
               
        if compl.find('use'):
            car_compl.append(compl.find('td').get_text())
        
        
    
    
    return doors, name, car_compl
    
    
    

In [17]:
#content = get_page_content('https://moscow.drom.ru/bmw/x5/38266717.html')
#print(parse_content(content))

(None, 'BMW', 'черный', 'дизель', '3', 'xDrive 25d AT  Business', '5', '2017', None, 'автомат', '3.0', '218', 'Дополнительно: Автомобиль в идеальном состоянии, ВЕСЬ в родной краске, никаких вложений не требует, 500 км назад было сделано ТО, ПТС оригинал, готов на любые проверки.', '102000', ['Зимняя комплектация', 'Нагнетатель', 'Система старт-стоп', 'Поддержка ручного переключения передач', 'Электроусилитель руля', 'Изменение передаточного отношения руля (VGS/VGRS)', 'Пневматическая подвеска', 'Алюминиевые колесные диски', 'Самонесущие шины', 'Система рекуперативного торможения', 'Биксеноновые фары', 'Светодиодные фары', 'Светодиодные ходовые огни', 'Омыватели передних фар', 'Передние противотуманные фары', 'Cветодиодные задние фонари', 'Дополнительный стоп-сигнал', 'Датчик света', 'Датчики дождя', 'Боковые зеркала с повторителями указателей поворота', 'Электропривод боковых зеркал', 'Электроподогрев зеркал', 'Ручки дверей в цвет кузова', 'Электропривод багажника', 'Остекление с защит

In [32]:
with open("bmws_1.csv", "wt", newline='') as output_file:
    writer = csv.writer(output_file, delimiter=",")
    writer.writerow(('bodyType','brand','color','fuelType','modelDate','name','numberOfDoors','productionDate',
                     'vehicleConfiguration','vehicleTransmission','engineDisplacement','enginePower','description',
                     'mileage','Комплектация','Привод','Руль','Состояние','Владельцы','ПТС','Таможня','Владение','price', 'model'))
    
    with open('bmw_links_1.txt') as links_file:
        for link in links_file:
            print('Parsing: ', link)

            page_content = get_page_content(link)
            
            if page_content == None:
                continue
            
            parsed_content = None
            
            try_num = 0;
            while parsed_content == None:
                
                if try_num >= 15:
                    break;
                
                try:
                    try_num += 1
                    page_content = get_page_content(link)
                    parsed_content = parse_content(page_content)
                except:
                    pass
                finally:
                    print("Try number: ", try_num)
            
            if try_num >= 15:
                continue
                    
            print(parsed_content)

            writer.writerow(parsed_content)

Parsing:  https://moscow.drom.ru/bmw/5-series/38768916.html

Try number:  1
('седан', 'BMW', 'черный', 'дизель', '6', '520d AT', '4', '2013', None, 'автомат', '2.0', '184', None, '96000', ['Нагнетатель', 'Система старт-стоп', 'Электроусилитель руля', 'Спортивная подвеска', 'Алюминиевые колесные диски', 'Датчики дождя', 'Электропривод боковых зеркал', 'Электроподогрев зеркал', 'Кнопка запуска двигателя', 'Тканевая обивка салона', 'Электрические стеклоподъемники передние', 'Электрические стеклоподъемники задние', 'Подушка безопасности водительская', 'Подушка безопасности переднего пассажира', 'Подушка безопасности пассажира с функцией деактивации', 'Подушка безопасности боковая', 'Крепление ISOFIX', 'Антиблокировочная система (ABS)', 'Антипробуксовочная система (TCS)', 'Круиз-контроль', 'Климат-контроль', 'Радио', 'CD-плейер', 'Полноцветный ЖК-монитор', 'Бортовой компьютер'], 'задний', 'левый', None, '4', None, None, None, '1220000', '5-Series')
Parsing:  https://moscow.drom.ru/bmw/m5/38

Try number:  1
(None, 'BMW', 'белый', 'бензин', '3', None, None, '2014', None, 'автомат', '3.0', '306', None, '101978', None, '4WD', 'левый', None, '2', None, None, None, '2150000', 'X5')
Parsing:  https://tula.drom.ru/bmw/3-series/38808996.html

Try number:  1
('седан', 'BMW', 'синий', 'бензин', '5', None, None, '2010', None, 'автомат', '2.0', '136', None, '108500', None, 'задний', 'левый', None, '2', None, None, None, '680000', '3-Series')
Parsing:  https://moscow.drom.ru/bmw/1-series/38807775.html

Try number:  1
('хэтчбек', 'BMW', 'черный', 'бензин', '1', '118i AT Base', '5', '2008', None, 'автомат', '2.0', '136', None, '152000', ['Гидроусилитель руля', 'Спортивная подвеска', 'Заниженная подвеска', 'Алюминиевые колесные диски', 'Самонесущие шины', 'Галогенные фары', 'Биксеноновые фары', 'Передние противотуманные фары', 'Задние противотуманные фонари', 'Дополнительный стоп-сигнал', 'Датчики дождя', 'Боковые зеркала в цвет кузова', 'Электропривод боковых зеркал', 'Электроподогрев зер

Try number:  1
(None, 'BMW', 'серебристый', 'дизель', '3', '730d AT', '4', '2000', None, 'автомат', '2.9', '184', None, '314000', ['Нагнетатель', 'Поддержка ручного переключения передач', 'Электрогидроусилитель руля', 'Алюминиевые колесные диски', 'Ксеноновые фары', 'Омыватели передних фар', 'Передние противотуманные фары', 'Задние противотуманные фонари', 'Дополнительный стоп-сигнал', 'Датчики дождя', 'Боковые зеркала в цвет кузова', 'Электроподогрев зеркал', 'Доводчик двери', 'Ручки дверей в цвет кузова', 'Электропривод багажника', 'Регулируемый люк', 'Кожаная оплетка рулевого колеса', 'Регулируемая рулевая колонка', 'Телескопическая регулировка руля', 'Вертикальная регулировка руля', 'Управление аудиосистемой на руле', 'Тахометр', 'Датчик наружной температуры', 'Кожаная обивка салона', 'Разделительный подлокотник спереди', 'Разделительный подлокотник сзади', 'Карманы на спинках передних сидений', 'Электропривод водительского сиденья', 'Электропривод передних сидений', 'Электроподогр

Try number:  1
(None, 'BMW', 'белый', 'бензин', '2', None, None, '2008', None, 'автомат', '3.0', None, None, '196000', None, '4WD', 'левый', None, '3', None, None, None, '780000', 'X5')
Parsing:  https://moscow.drom.ru/bmw/x5/37706345.html

Try number:  1
Try number:  2
Try number:  3
Try number:  4
Try number:  5
Try number:  6
Try number:  7
Try number:  8
Try number:  9
Try number:  10
Try number:  11
Try number:  12
Try number:  13
Try number:  14
Try number:  15
Parsing:  https://klin.drom.ru/bmw/5-series/38411753.html

Try number:  1
('седан', 'BMW', 'черный', 'бензин', '3', '525i AT', '4', '1990', None, 'автомат', '2.5', '170', None, '340000', ['Гидроусилитель руля', 'Алюминиевые колесные диски', 'Система рекуперативного торможения', 'Галогенные фары', 'Ксеноновые фары', 'Боковые зеркала в цвет кузова', 'Электропривод боковых зеркал', 'Электроподогрев зеркал', 'Заводская тонировка', 'Обогрев заднего стекла', 'Люк', 'Рейлинги', 'Кожаная оплетка рулевого колеса', 'Управление аудио

Try number:  1
(None, 'BMW', 'синий', 'бензин', '2', None, None, '2008', None, 'автомат', '3.0', '272', None, '180000', None, '4WD', 'левый', None, None, None, None, None, '850000', 'X5')
Parsing:  https://moscow.drom.ru/bmw/x6/38802728.html

Try number:  1
(None, 'BMW', 'черный', 'бензин', '2', None, None, '2016', None, 'автомат', '3.0', '306', None, '72000', None, '4WD', 'левый', None, '1', None, None, None, '2778000', 'X6')
Parsing:  https://moscow.drom.ru/bmw/x2/38802679.html

Try number:  1
(None, 'BMW', 'серый', 'бензин', '1', 'sDrive 18i DCT Advantage', '5', '2018', None, 'автомат', '1.5', '140', None, '17500', ['Левый руль', 'Нагнетатель', 'Система старт-стоп', 'Поддержка ручного переключения передач', 'Электроусилитель руля', 'Спортивная подвеска', 'Алюминиевые колесные диски', 'Самонесущие шины', 'Галогенные фары', 'Светодиодные ходовые огни', 'Омыватели передних фар', 'Передние противотуманные фары', 'Cветодиодные задние фонари', 'Дополнительный стоп-сигнал', 'Датчик света',

...

Try number:  1
('седан', 'BMW', 'серебристый', 'бензин', '5', '525i AT', '4', '2003', None, 'автомат', '2.5', '192', None, '267000', ['Левый руль', 'Гидроусилитель руля', 'Передний стабилизатор', 'Задний стабилизатор', 'Алюминиевые колесные диски', 'Галогенные фары', 'Ксеноновые фары', 'Передние противотуманные фары', 'Датчики дождя', 'Боковые зеркала в цвет кузова', 'Электропривод боковых зеркал', 'Электроподогрев зеркал', 'Окраска кузова металлик', 'Кожаная оплетка рулевого колеса', 'Регулируемая рулевая колонка', 'Телескопическая регулировка руля', 'Вертикальная регулировка руля', 'Управление аудиосистемой на руле', 'Тканевая обивка салона', 'Кожаная обивка салона', 'Разделительный подлокотник спереди', 'Разделительный подлокотник сзади', 'Электропривод водительского сиденья', 'Электроподогрев передних сидений', 'Память расположения сиденья', 'Центральный замок', 'Ключ ДУ (дистанционный ключ)', 'Электрические стеклоподъемники передние', 'Электрические стеклоподъемники задние', 'Паке

Try number:  1
(None, 'BMW', 'белый', 'дизель', '3', None, None, '2018', None, 'автомат', '3.0', '313', None, '39000', None, '4WD', 'левый', None, None, None, None, None, '2900000', 'X5')
Parsing:  https://moscow.drom.ru/bmw/5-series/38453886.html

Try number:  1
('седан', 'BMW', 'серый', 'бензин', None, None, None, '1996', None, 'механика', '2.5', '190', None, '570000', None, 'задний', 'левый', None, '7', None, None, None, '130000', '5-Series')
Parsing:  https://moscow.drom.ru/bmw/x6/37638875.html

Try number:  1
(None, 'BMW', 'черный', 'дизель', '2', 'xDrive M50d AT', '5', '2016', None, 'автомат', '3.0', '381', None, '16200', ['Зимняя комплектация', 'Нагнетатель', 'Система старт-стоп', 'Поддержка ручного переключения передач', 'Изменение передаточного отношения руля (VGS/VGRS)', 'Электрогидроусилитель руля', 'Пневматическая подвеска', 'Алюминиевые колесные диски', 'Самонесущие шины', 'Биксеноновые фары', 'Светодиодные фары', 'Светодиодные ходовые огни', 'Омыватели передних фар', 'Пер

Try number:  1
('седан', 'BMW', 'белый', 'дизель', '7', None, None, '2019', None, 'автомат', '2.0', '190', None, '21000', None, '4WD', 'левый', None, '2', None, None, None, '2590000', '5-Series')
Parsing:  https://moscow.drom.ru/bmw/x5/38094605.html

Try number:  1
Try number:  2
Try number:  3
Try number:  4
Try number:  5
Try number:  6
Try number:  7
Try number:  8
Try number:  9
Try number:  10
Try number:  11
Try number:  12
Try number:  13
Try number:  14
Try number:  15
Parsing:  https://moscow.drom.ru/bmw/4-series/38094397.html

Try number:  1
('купе', 'BMW', 'белый', 'бензин', '1', '430i AT xDrive Base', '2', '2018', None, 'автомат', '2.0', '249', None, '45400', ['Зимняя комплектация', 'Нагнетатель', 'Система старт-стоп', 'Поддержка ручного переключения передач', 'Электроусилитель руля', 'Изменение передаточного отношения руля (VGS/VGRS)', 'Спортивная подвеска', 'Алюминиевые колесные диски', 'Самонесущие шины', 'Система рекуперативного торможения', 'Светодиодные фары', 'Светод

Try number:  1
('седан', 'BMW', 'черный', 'бензин', '7', None, None, '2019', None, 'автомат', '2.0', '184', None, '37386', None, 'задний', 'левый', None, '2', None, None, None, '2145000', '5-Series')
Parsing:  https://moscow.drom.ru/bmw/5-series/38445499.html

Try number:  1
('седан', 'BMW', 'серый', 'бензин', '6', None, None, '2011', None, 'автомат', '2.0', '184', None, '127452', None, 'задний', 'левый', None, '5', None, None, None, '829000', '5-Series')
Parsing:  https://moscow.drom.ru/bmw/5-series/37507989.html

Try number:  1
('седан', 'BMW', 'бежевый', 'бензин', '6', None, None, '2012', None, 'автомат', '2.0', '245', None, '118701', None, 'задний', 'левый', None, '3', None, None, None, '890000', '5-Series')
Parsing:  https://moscow.drom.ru/bmw/5-series/37460502.html

Try number:  1
('седан', 'BMW', 'черный', 'дизель', '7', None, None, '2018', None, 'автомат', '2.0', '190', None, '29820', None, '4WD', 'левый', None, '1', None, None, None, '2360000', '5-Series')
Parsing:  https://mo

Try number:  1
Try number:  2
Try number:  3
Try number:  4
Try number:  5
Try number:  6
Try number:  7
Try number:  8
Try number:  9
Try number:  10
Try number:  11
Try number:  12
Try number:  13
Try number:  14
Try number:  15
Parsing:  https://moscow.drom.ru/bmw/x3/38642059.html

Try number:  1
Try number:  2
Try number:  3
Try number:  4
Try number:  5
Try number:  6
Try number:  7
Try number:  8
Try number:  9
Try number:  10
Try number:  11
Try number:  12
Try number:  13
Try number:  14
Try number:  15
Parsing:  https://moscow.drom.ru/bmw/x7/38624472.html

Try number:  1
Try number:  2
Try number:  3
Try number:  4
Try number:  5
Try number:  6
Try number:  7
Try number:  8
Try number:  9
Try number:  10
Try number:  11
Try number:  12
Try number:  13
Try number:  14
Try number:  15
Parsing:  https://moscow.drom.ru/bmw/x7/38624164.html

Try number:  1
Try number:  2
Try number:  3
Try number:  4
Try number:  5
Try number:  6
Try number:  7
Try number:  8
Try number:  9
Try num